In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [5]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time']=encoder.fit_transform(df['time'])   

In [6]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [7]:
df['time'].value_counts()

0    176
1     68
Name: time, dtype: int64

In [8]:
x = df.drop(labels=['time'],axis=1)
y = df['time']

In [9]:
x

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [10]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=42)

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [13]:
categorical_cols = ['sex','smoker','day']
numerical_cols = ['total_bill','tip','size']

In [14]:
#Feature Engineering Automation
num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    steps = [
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('onehotencoder',OneHotEncoder())
    ]
)

In [15]:
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [16]:
x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
models = {
    'Random Forest':RandomForestClassifier()
}

In [19]:
from sklearn.metrics import accuracy_score

In [20]:
def evaluate_model(x_train,y_train,x_test,y_test,models):

    report={}
    
    for i in range(len(models)):
        
        model = list(models.values())[i]
        
        #Train model
        
        model.fit(x_train,y_train)
        
        #Predict testing data
        
        y_test_pred = model.predict(x_test)

        #Get accuracy for test data prediction

        test_model_score = accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]] = test_model_score

        return report
        

In [21]:
evaluate_model(x_train,y_train,x_test,y_test,models)

{'Random Forest': 0.9591836734693877}

In [22]:
classifier = RandomForestClassifier()

In [23]:
#Hyperparameter tuning
parameter = {'max_depth':[3,5,10,None],
             'n_estimators':[100,200,300],
             'criterion':['gini','entropy']
             }

In [24]:
from sklearn.model_selection import RandomizedSearchCV

In [25]:
cv = RandomizedSearchCV(classifier,param_distributions=parameter,scoring='accuracy',cv=5,verbose=3)
cv.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.974 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.923 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=100;, score=1.000 total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.949 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.923 total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.974 total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.923 total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.974 total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.923 total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.949 total ti

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [26]:
cv.best_params_

{'n_estimators': 100, 'max_depth': None, 'criterion': 'gini'}

In [27]:
classifier = RandomForestClassifier(n_estimators=200,max_depth=3,criterion='entropy')
classifier.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=3, n_estimators=200)

In [28]:
y_pred = classifier.predict(x_test)

In [29]:
def evaluate_model(x_train,y_train,x_test,y_test,models):

    report={}
    
    for i in range(len(models)):
        
        model = list(models.values())[i]
        
        #Train model
        
        model.fit(x_train,y_train)
        
        #Predict testing data
        
        y_test_pred = model.predict(x_test)

        #Get accuracy for test data prediction

        test_model_score = accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]] = test_model_score

        return report

In [30]:
evaluate_model(x_train,y_train,x_test,y_test,models)

{'Random Forest': 0.9591836734693877}